## Renault Zoé sur le marché de l'occasion

L'objectif est de générer un fichier de données sur le prix des Renault Zoé sur le marché de l'occasion en Ile de France, PACA et Aquitaine. 
Vous utiliserez leboncoin.fr comme source. Si leboncoin ne fonctionne plus vous pouvez vous rabattre sur d'autres sites d'annonces comme lacentrale, paruvendu, autoplus,... Le fichier doit être propre et contenir les infos suivantes : version ( il y en a 3), année, kilométrage, prix, téléphone du propriétaire, est ce que la voiture est vendue par un professionnel ou un particulier.
Vous ajouterez une colonne sur le prix de l'Argus du modèle que vous récupérez sur ce site http://www.lacentrale.fr/cote-voitures-renault-zoe--2013-.html.

Les données quanti (prix, km notamment) devront être manipulables (pas de string, pas d'unité).
Vous ajouterez une colonne si la voiture est plus chere ou moins chere que sa cote moyenne.﻿

In [78]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from selenium import webdriver
import math

In [3]:
def get_soup_from_link(link):
    hdr = {"Accept-Language": "en-US,en;q=0.5",
           "User-Agent": 'Mozilla/5.0',
           "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "Referer": "http://thewebsite.com",
           "Connection": "keep-alive"}
    req = requests.get(link, headers=hdr)
    soup = BeautifulSoup(req.content, 'html.parser')
    return soup

In [63]:
def get_info_from_link(link):
    soup = get_soup_from_link(link)
    title = soup.find('h1', class_='_1KQme').text
    version = get_version_from_title(title)
    sub = soup.findAll('div', class_='_3Jxf3')
    annee = int(sub[2].text.replace(" ", ""))
    km = sub[3].text.replace(" ", "")[:-2]
    prix = soup.find('span', class_='_1F5u3').text.replace(" ", "")[:-1]
    pro = soup.findAll('div', {'data-qa-id':'storebox_container'}) is not None
    return [version, annee, km, prix, pro]

In [86]:
def get_phone_from_link_list(link_list):
    liste = []
    driver = webdriver.Firefox()
    for links in link_list:
        driver.get(links)
        element = driver.find_element_by_css_selector('._2sPVF')
        element.click()
        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        liste = liste + [soup.find('a', class_ = '_2sNbI ObuDQ GXQkc _2BP2c').text]
    driver.quit()
    return liste

In [6]:
def get_list_of_urls_from_region(region, npage):
    liste = []
    for page in range(1, npage) :
        soup = get_soup_from_link(search_url.format(region, str(page)))
        all_a = soup.find('div', class_ = 'react-tabs__tab-panel react-tabs__tab-panel--selected').findAll('a')
        liste = liste + list(map(lambda x : x.get('href'), all_a))
    return liste

In [14]:
def get_version_from_title(title) :
    founds = [word for word in title.lower().split() if word in modeles]
    return founds[0] if len(founds) == 1 else np.nan

In [8]:
def get_cotes() :
    title = []
    annee = []
    prix = []
    
    for year in range(2012, 2018) :
        lcurl = curl.format("cote-voitures-renault-zoe--{}-.html").format(str(year))
        listing = get_soup_from_link(lcurl).findAll('div', class_ = "listingResultLine")
        liste_cote = list(map(lambda x : x.find('a').get('href'), listing))
        for item in liste_cote :
            soup = get_soup_from_link(curl.format(item))
            title = title +  [soup.find('span', class_ = 'sizeC clear txtGrey7C sizeC').text]
            annee = annee + [year]
            prix_attr = soup.find('span', class_ = "jsRefinedQuot")
            prix = prix + [prix_attr.text if prix_attr is not None else None]
    df = pd.DataFrame({'title' : title, 'annee' : annee, 'prix' : prix})
    return df

In [46]:
modeles = ['life', 'zen', 'intens', 'edition one']

main_url = "https://www.leboncoin.fr{}"
search_url = "https://www.leboncoin.fr/recherche/?category=2&text=renault%20zoe&regions={}&page={}"

curl = "https://www.lacentrale.fr/{}"

In [64]:
result = pd.DataFrame(columns=['version', 'annee', 'km', 'prix', 'pro'])
liste_idf = list(map(lambda x : main_url.format(x), get_list_of_urls_from_region('12', 5)))

for link in liste_idf :
    value = pd.Series(get_info_from_link(link), index=['version', 'annee', 'km', 'prix', 'pro'])
    result = result.append(value, ignore_index=True)
result = result.dropna()
result[["annee", "km", "prix"]] = result[["annee", "km", "prix"]].astype(int)

In [89]:
phones = get_phone_from_link_list([liste_idf[0]])

In [90]:
phones

['01 86 65 50 16']

In [79]:
df = get_cotes()
df["version"] = df["title"].apply(get_version_from_title).astype(str)
df["cote"] = df["prix"].apply(lambda x : int(x.replace(" ", "")))
df = df.dropna()
cotes = df.groupby(["version", "annee"], as_index=False).mean()
cotes['cote'] = cotes['cote'].apply(lambda x : math.floor(x))

In [85]:
final = pd.merge(result, cotes, how='inner', on=["version", "annee"])
final["IsLessThanCote"] = final.prix < final.cote
final

,version,annee,km,prix,pro,cote,IsLessThanCote
0,intens,2017,7828,13990,True,15116,True
1,intens,2017,100,15490,True,15116,False
2,intens,2017,3500,19000,True,15116,False
3,intens,2015,26578,9990,True,11819,True
4,intens,2015,37535,9990,True,11819,True
5,intens,2015,71609,8290,True,11819,True
6,intens,2015,28240,9800,True,11819,True
7,intens,2015,25047,11490,True,11819,True
8,intens,2015,28755,9990,True,11819,True
9,intens,2015,28755,9990,True,11819,True
